In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [27]:
df = pd.read_pickle('../sparse_ratings.pkl')

In [39]:
df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,330966,330967,330968,330969,330970,330971,330972,330973,330974,330975
movieId,,,,,,,,,,,,,,,,,,,,,
1203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
527,0.0,5.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,...,0.0,4.0,0.0,0.0,0.0,4.5,4.0,0.0,4.5,3.5
1193,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,3.0
750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
movies_info = pd.read_csv("../suitable_movies.csv")

In [29]:
movies_info.set_index("movieId", inplace=True)

In [33]:
def filter_users(df=df, n=1, categories=4):
    movie_genres = movies_info['suitable_genre']
    df_ = df.copy()
    valid_users = []
    for user in df_.columns:
        genres = {
            "Action": 0,
            "Adventure": 0,
            "Animation": 0,
            "Comedy": 0,
            "Crime": 0,
            "Drama": 0,
            "Fantasy": 0,
            "Horror": 0,
            "Mystery": 0,
            "Romance": 0,
            "Sci-Fi": 0,
            "Thriller": 0
        }
        user_data = df_[df_[user] != 0.0][user]
        movie_genres = movies_info[movies_info.index.isin(user_data.index)]['suitable_genre']
        for movie_genre in movie_genres:
            genres[movie_genre] += 1
        #print(genres)
        one_more_than_n = False
        valid_user_counter = 0
        for genre, count in genres.items():
            if count >= n:
                valid_user_counter += 1
            if count > n: one_more_than_n = True
        # if the user has at least n movies in each category, and they have at least one movie to be predicted - we say they are valid for our purposes, and we store their id (column name of the df_)
        if valid_user_counter >= categories and one_more_than_n:
            valid_users.append(user)
            print(f'found a valid user - {user}')
    return valid_users

In [34]:
from sklearn.utils import shuffle


def filter_ratings(valid_users, df=df, n=2, categories=4):
    filtered = []
    for user_id in valid_users:
        genres = {
            "Action": 0,
            "Adventure": 0,
            "Animation": 0,
            "Comedy": 0,
            "Crime": 0,
            "Drama": 0,
            "Fantasy": 0,
            "Horror": 0,
            "Mystery": 0,
            "Romance": 0,
            "Sci-Fi": 0,
            "Thriller": 0
        }
        user_ratings = df[user_id][df[user_id] != 0.0]
        user_genres = movies_info[movies_info.index.isin(user_ratings.index)]['suitable_genre']
        #looks like double work but here we are ensuring only the genres that were rated at least n times are used
        for movie_genre in user_genres:
            genres[movie_genre] += 1
        unique_user_genres = [key for key, value in genres.items() if value >= n]
        # then making sure that we are picking the genres randomly (we will cut it later)
        user_genres = shuffle(user_genres)
        # for every unique genre the user has rated - pick n random movies
        lst_filtered_movies = []
        lst_filtered_out_movies = []
        # making sure every genre is represented the same amount
        for genre in unique_user_genres[:categories]:
            #print(genre)
            movies_of_genre = user_genres[user_genres == genre].index.values
            df_movies_of_genre = shuffle(pd.DataFrame(data=movies_of_genre))
            #print(df_movies_of_genre[:n])
            lst_filtered_movies.append(df_movies_of_genre[:n])
            
        lst_filtered_movies = pd.concat(lst_filtered_movies).values
        for movie_id in user_ratings.index.values:
            if movie_id not in lst_filtered_movies:
                lst_filtered_out_movies.append(movie_id)
        filtered.append([user_id, [x[0] for x in lst_filtered_movies], lst_filtered_out_movies])
    return_df = pd.DataFrame(data=filtered, columns=['userId', 'testMovies', 'evalMovies'])

    return return_df.set_index('userId')

In [ ]:
import time
sample_size = 100000
all_ns = [i for i in range(1, 5)]
all_cats = [i for i in range(4, 12)]
for n in all_ns:
    beginning = time.time()
    print(beginning)
    for cat in all_cats:
        df_loop = shuffle(df.iloc[:,:sample_size].T).T
        valid_users = filter_users(df=df_loop, n=n, categories=cat)
        print('found all valid users')
        eval_df = filter_ratings(valid_users=valid_users, df=df_loop, n=n, categories=cat)
        eval_df.to_pickle(f'n{n}_c{cat}')
        print(f'n{n}_c{cat}: {eval_df}')
        print(f'time: {time.time() - beginning}')

1714566908.5511053
found a valid user - 34182
found a valid user - 58775
found a valid user - 21520
found a valid user - 34894
found a valid user - 101428
found a valid user - 38800
found a valid user - 87463
found a valid user - 64664
found a valid user - 47709
found a valid user - 80614
found a valid user - 24465
found a valid user - 102729
found a valid user - 23540
found a valid user - 87690
found a valid user - 24508
found a valid user - 52691
found a valid user - 6645
found a valid user - 108238
found a valid user - 34671
found a valid user - 91067
found a valid user - 70344
found a valid user - 47661
found a valid user - 60298
found a valid user - 29006
found a valid user - 15376
found a valid user - 84041
found a valid user - 38107
found a valid user - 88109
found a valid user - 38949
found a valid user - 21014
found a valid user - 37239
found a valid user - 42884
found a valid user - 81672
found a valid user - 100551
found a valid user - 73476
found a valid user - 90882
found 